In [2]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px

# Load the preprocessed data
# wine_df = pd.read_csv('preprocessed_wine_data.csv')

external_stylesheets = [
    {
        "href": "assets/style.css",        
        "rel": "stylesheet",
    },
]


TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "The Story of Wine: From Grapes to Glass!"